In [2]:
# General Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import defaultdict
import pickle
from PIL import Image
import urllib
from adjustText import adjust_text


In [3]:
chromedriver_autoinstaller.install()

import warnings
warnings.filterwarnings('ignore')

## Pull Data

In [4]:
chromedriver_autoinstaller.install()

if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640902/Live/England-Premier-League-2022-2023-Chelsea-Fulham"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

Region: England, League: Premier League, Season: 2022/2023, Match Id: 1640902


In [5]:
homeID = home_data['teamId']
awayID = away_data['teamId']

names = {
    homeID : home_name, 
    awayID : away_name
}

## Colours 

In [6]:
home_color = '#2D5DE7'
away_color = '#9C9C9C'

## Get Players
- Both 11 with most minutes and all players home and away

In [7]:
# For the home team
team_players = events_df[events_df.teamId == homeID].groupby('playerName').agg({'minute': [min, max]}).reset_index()
team_players = pd.concat([team_players['playerName'], team_players['minute']], axis=1)
team_players['mins_played'] = team_players['max'] - team_players['min']
team_players = team_players.sort_values('mins_played', ascending=False)
team_players_names = team_players.playerName
team_players_names_top11 = team_players.playerName[:11].tolist()


# For the away team
team2_players = events_df[events_df.teamId == awayID].groupby('playerName').agg({'minute': [min, max]}).reset_index()
team2_players = pd.concat([team2_players['playerName'], team2_players['minute']], axis=1)
team2_players['mins_played'] = team2_players['max'] - team2_players['min']
team2_players = team2_players.sort_values('mins_played', ascending=False)
team2_players_names = team2_players.playerName
team2_players_names_top11 = team2_players.playerName[:11].tolist()


# All Players
team_all_names = team_players_names.tolist()
team2_all_names = team2_players_names.tolist()

all_players = team_all_names + team2_all_names
all_players

['Benoît Badiashile',
 'Thiago Silva',
 'Enzo Fernández',
 'Conor Gallagher',
 'Kai Havertz',
 'Kepa Arrizabalaga',
 'Marc Cucurella',
 'Mason Mount',
 'Reece James',
 'Hakim Ziyech',
 'Noni Madueke',
 'Mykhailo Mudryk',
 'César Azpilicueta',
 'Raheem Sterling',
 'David Datro Fofana',
 'Ben Chilwell',
 'Bernd Leno',
 'João Palhinha',
 'Andreas Pereira',
 'Antonee Robinson',
 'Issa Diop',
 'Kenny Tete',
 'Harrison Reed',
 'Tim Ream',
 'Aleksandar Mitrovic',
 'Bobby De Cordova-Reid',
 'Willian',
 'Harry Wilson',
 'Manor Solomon',
 'Carlos Vinícius',
 'Tom Cairney',
 'Layvin Kurzawa']

## Data Engineering - xA
- Just shifting index by one won't capture actions that should qualify as a potential assist and ergo xA values. 
- E.g. if a player carries the ball/takes a 'ball touch' then the preceding action won't qualify as an assist if we just shift the index by one
- Hence we're going to get all key passes for each player from the events_df dataframe, then just iterate through subsequent indexes until we get the first shot 
- Then just merge the id on the xG df

In [8]:
keyPassCols = ['keyPassLong', 
               'keyPassShort', 
               'keyPassCross',
               'keyPassCorner', 
               'keyPassThroughball', 
               'keyPassFreekick', 
               'keyPassThrowin', 
               'keyPassOther'
              ]

shot_types = ['SavedShot', 
              'MissedShots', 
              'Goal', 
              'ShotOnPost'
             ]

df_KP = pd.DataFrame(columns=['playerAssist', 'id'])

for player in all_players:
    index = []
    for col in keyPassCols:
        idxs = events_df[(events_df['playerName'] == player) &
                         (events_df[col] == 1)].index.values.tolist()
        for idx in idxs:
            if idx not in index:
                index.append(idx)

    # Now need to get correspoding id of subsequent shot, then just merge on the xA dataframe

    for idx in index:
        df = events_df.loc[idx+1:]

        for idx, row in df.iterrows():
            if any(row['type'] == x for x in shot_types):
                df_KP.loc[len(df_KP)] = [player, row['id']]
                break

            

df_KP

,playerAssist,id
0,Benoît Badiashile,2.503431e+09
1,Enzo Fernández,2.503433e+09
2,Conor Gallagher,2.503468e+09
3,Conor Gallagher,2.503476e+09
4,Marc Cucurella,2.503417e+09
5,Hakim Ziyech,2.503440e+09
6,Noni Madueke,2.503472e+09
7,Noni Madueke,2.503481e+09
8,Ben Chilwell,2.503481e+09
9,João Palhinha,2.503423e+09


## XG Model
- Run shot data through our xG model, so we can get the xA and xG for individual players (and how they rank relative to distribution)
- First run through function which reduces it to the necessary form (described in detail in our xG model notebook)

In [9]:
pitch_length_x = 100
pitch_length_y = 100

feature_cols = ['id',
                'teamId',
                'prevTeam',
                'x', 
                'y',
                'playerName', 
                'playerAssist',
                'minute', 
                'second',
                'type',
                'prevEvent',
                'isGoal',
                'shotRightFoot',
                'shotLeftFoot',
                'shotHead',
                'shotOpenPlay',
                'shotSetPiece',
                'shotCounter', 
#                 'shotDirectCorner',
                'penaltyScored',
                'penaltyMissed',
                'goalOwn',
#                 'shotLongPass',
#                 'shotShortPass',
#                 'shotCross',
#                 'shotThroughBall',
                'bigChanceScored',
                'bigChanceMissed'
              ]

shot_types = ['SavedShot', 
              'MissedShots', 
              'Goal', 
              'ShotOnPost'
             ]


def shotData(df):
    df["passRecipient"] = df['playerName'].shift(-1)
    df['teamRecipient'] = df['teamId'].shift(-1)
    df['prevEvent'] = df['type'].shift(1)
    df['shotLongPass'] = df['keyPassLong'].shift(1)
    df['shotShortPass'] = df['keyPassShort'].shift(1)
    df['shotCross'] = df['keyPassCross'].shift(1)
    df['shotThroughBall'] = df['keyPassThroughball'].shift(1)
    df['playerAssist'] = df['playerName'].shift(1)
    df['prevTeam'] = df['teamId'].shift(1)

    column_to_move = df.pop("passRecipient")
    df.insert(29, "passRecipient", column_to_move)
    column_to_move = df.pop("teamRecipient")
    df.insert(27, "teamRecipient", column_to_move)
    column_to_move = df.pop('prevEvent')
    df.insert(11, "prevEvent", column_to_move)

    df_shots = df[df['type'].isin(shot_types)].reset_index(drop=True)[feature_cols]

    df_shots = df_shots[(df_shots['penaltyScored'] == False) & 
                        (df_shots['penaltyMissed'] == False) &
                        (df_shots['goalOwn'] == False)
                       ]
#     df_shots['shotTakeOn'] = df_shots['prevEvent'] == 'TakeOn'
#     df_shots['shotDefensiveAction'] = df_shots['prevEvent'].isin(['Challenge', 'Tackle', 'Interception', 'Clearance',
#                                                                   'BallRecovery', 'BlockedPass'])

    df_shots['distance_to_goal'] = np.sqrt(((pitch_length_x - df_shots['x'])**2 + 
                                    (pitch_length_y - df_shots['y'])**2))
    df_shots['distance_to_center'] = abs(df_shots['y'] - pitch_length_y/2)
    df_shots['angle'] = np.absolute(np.degrees(np.arctan((abs((pitch_length_y/2) - df_shots['y'])) 
                                                         / (pitch_length_x - df_shots['x']))))


    df_shots = df_shots.drop(['prevEvent', 'penaltyScored', 'penaltyMissed', 'goalOwn', 'type'], axis=1)
    
    # Add Big Chance Column
    df_shots['bigChance'] = df_shots['bigChanceMissed'] | df_shots['bigChanceScored']
    df_shots = df_shots.drop(['bigChanceMissed', 'bigChanceScored'], axis=1)

    cat_cols = ['shotRightFoot',
            'shotLeftFoot',
            'shotHead',
            'shotOpenPlay',
            'shotSetPiece',
#             'shotDirectCorner',
            'shotCounter',
#             'shotLongPass',
#             'shotShortPass',
#             'shotCross',
#             'shotThroughBall',
#             'shotTakeOn', 
#             'shotDefensiveAction',
            'bigChance'
           ]
    df_shots[cat_cols] *= 1
    
    obj_cols = [
#             'shotLongPass',
#             'shotShortPass',
#             'shotCross',
#             'shotThroughBall',
             ]
    
    for col in obj_cols:
        df_shots[col] = pd.to_numeric(df_shots[col])
    

    return df_shots 

In [10]:
data = shotData(events_df).reset_index(drop=True)
X_game = data.iloc[:, 10:]
Y_game = data['isGoal']
X_game

,shotRightFoot,shotLeftFoot,shotHead,shotOpenPlay,shotSetPiece,shotCounter,distance_to_goal,distance_to_center,angle,bigChance
0,0,0,1,1,0,0,64.282579,13.5,53.471145,0
1,1,0,0,1,0,0,37.890500,18.8,41.167053,0
2,1,0,0,1,0,0,63.751549,10.5,27.582029,0
3,1,0,0,1,0,0,55.139460,4.4,26.053495,1
4,0,1,0,1,0,0,49.285393,4.4,13.240520,0
5,1,0,0,0,0,1,71.316828,20.3,59.411265,0
6,0,1,0,1,0,0,36.744796,14.3,58.684020,1
7,1,0,0,1,0,0,71.198034,19.6,52.572949,0
8,1,0,0,1,0,0,48.208091,3.9,15.461218,0
9,0,1,0,1,0,0,37.079914,18.4,43.484592,0


In [11]:
# Import our xG model 
xg_model = pickle.load(open('xgModel_XGBoost22.sav', 'rb'))

In [12]:
y_pred_game = xg_model.predict_proba(X_game)
df_pred_game = pd.DataFrame(y_pred_game, columns = ['prob_no_goal', 'prob_goal'])
df_pred_game = pd.merge(data, df_pred_game, left_index=True, right_index=True)

shots_to_merge = df_pred_game[['id','playerName', 'prob_goal', 'teamId']]
shots_to_merge = shots_to_merge.merge(df_KP, how='outer', on='id').fillna('None')
shots_to_merge

,id,playerName,prob_goal,teamId,playerAssist
0,2.503408e+09,Bobby De Cordova-Reid,0.027340,170,Kenny Tete
1,2.503417e+09,Mason Mount,0.013392,15,Marc Cucurella
2,2.503423e+09,Andreas Pereira,0.024755,170,João Palhinha
3,2.503430e+09,Kai Havertz,0.509998,15,None
4,2.503431e+09,Kai Havertz,0.095483,15,Benoît Badiashile
5,2.503433e+09,Hakim Ziyech,0.014011,15,Enzo Fernández
6,2.503440e+09,Kai Havertz,0.354188,15,Hakim Ziyech
7,2.503453e+09,Willian,0.040157,170,João Palhinha
8,2.503463e+09,Andreas Pereira,0.078473,170,Willian
9,2.503463e+09,João Palhinha,0.024118,170,None


## Get individual player xG and xA 
- Doing non-penalty xG

In [13]:
df_xg = (
          shots_to_merge.groupby('playerName').agg({'prob_goal':'sum', 'teamId':'first'})
                                              .reset_index()
)

df_xg['team'] = df_xg['teamId'].map(names)
df_xg_home = df_xg[df_xg['team'] == home_name].reset_index(drop=True)
df_xg_away = df_xg[df_xg['team'] == away_name].reset_index(drop=True)

df_xg_home

,playerName,prob_goal,teamId,team
0,Benoît Badiashile,0.059064,15,Chelsea
1,Conor Gallagher,0.036795,15,Chelsea
2,David Datro Fofana,0.263560,15,Chelsea
3,Enzo Fernández,0.012698,15,Chelsea
4,Hakim Ziyech,0.014011,15,Chelsea
5,Kai Havertz,0.959669,15,Chelsea
6,Mason Mount,0.191998,15,Chelsea
7,Raheem Sterling,0.039276,15,Chelsea
8,Thiago Silva,0.052167,15,Chelsea


In [14]:
df_xa = (
          shots_to_merge.groupby('playerAssist').agg({'prob_goal':'sum', 'teamId':'first'})
                                                .reset_index()
)

df_xa['team'] = df_xa['teamId'].map(names)
df_xa_home = df_xa[df_xa['team'] == home_name].reset_index(drop=True)
df_xa_home = df_xa_home[df_xa_home['playerAssist'] != 'None']

df_xa_away = df_xa[df_xa['team'] == away_name].reset_index(drop=True)
df_xa_away = df_xa_away[df_xa_away['playerAssist'] != 'None']

df_xa_home

,playerAssist,prob_goal,teamId,team
0,Ben Chilwell,0.059064,15,Chelsea
1,Benoît Badiashile,0.095483,15,Chelsea
2,Conor Gallagher,0.315727,15,Chelsea
3,Enzo Fernández,0.014011,15,Chelsea
4,Hakim Ziyech,0.354188,15,Chelsea
5,Marc Cucurella,0.013392,15,Chelsea
7,Noni Madueke,0.217882,15,Chelsea


In [15]:
df_xa_home.rename({'prob_goal':'prob_assist', 'playerAssist':'playerName'}, axis='columns', inplace=True)
df_home = df_xg_home.merge(df_xa_home, on='playerName', how='outer').fillna(0)[['playerName', 'prob_goal', 'prob_assist']]

df_xa_away.rename({'prob_goal':'prob_assist', 'playerAssist':'playerName'}, axis='columns', inplace=True)
df_away = df_xg_away.merge(df_xa_away, on='playerName', how='outer').fillna(0)[['playerName', 'prob_goal', 'prob_assist']]
df_away

,playerName,prob_goal,prob_assist
0,Aleksandar Mitrovic,0.003870,0.000000
1,Andreas Pereira,0.103229,0.000000
2,Bobby De Cordova-Reid,0.105533,0.000000
3,João Palhinha,0.144973,0.064912
4,Tom Cairney,0.042894,0.000000
5,Willian,0.040157,0.156666
6,Kenny Tete,0.000000,0.027340
7,Manor Solomon,0.000000,0.054410


## Plot 

In [41]:
fig, ax = plt.subplots(figsize = (20,15))
fig.set_facecolor("#2B2B2B")
ax.patch.set_facecolor("#2B2B2B")
mpl.rcParams['figure.dpi'] = 800


# Home Team
plt.scatter(df_home.prob_assist, df_home.prob_goal, color=home_color, s=40)

text = []
for i, row in df_home.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['prob_assist']
    y = row['prob_goal']
    sf = float(x+y)
    t = plt.text(x = x+0.003, y = y+0.005, s = p1.upper(), 
                 bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(13, min(40, 70*(x+y))))
    text.append(t)
    

# Away Team
plt.scatter(df_away.prob_assist, df_away.prob_goal, color=away_color, s=40)

for i, row in df_away.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['prob_assist']
    y = row['prob_goal']
    sf = float(x+y)
    t = plt.text(x = x+0.003, y = y+0.005, s = p1.upper(), 
                 bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(13, min(40, 70*(x+y))))
    text.append(t)
    
plt.title('\nExpected Goals Contributions\n\n'.upper(), 
          loc='left', color='w', fontname = 'Sans Serif', fontsize = 30, y=1)
plt.suptitle(t = home_name.upper() + ' ' +score + ' ' + away_name.upper() + '  |  ' + date_str.upper() + '  |  '  + 'CREATED BY ...', x=0.125, y=0.93, ha='left', 
             fontsize=24, color='w')
plt.xlabel('xA', color='w', fontsize=30)
plt.ylabel('xG', color='w', fontsize=30)

ax.set_axisbelow(True)
ax.yaxis.grid(color='#848484', linestyle='dashed')
ax.xaxis.grid(color='#848484', linestyle='dashed')
# spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('w')
ax.spines['bottom'].set_color('w')
# ticks
for tick in ax.xaxis.get_ticklines(): 
    tick.set_color('w')
for tick in ax.yaxis.get_ticklines(): 
    tick.set_color('w')
plt.xticks(color='w')
plt.yticks(color='w')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

adjust_text(text, ax=ax)


fig.savefig('xg_vs_xa.png', 
            bbox_inches="tight",
            edgecolor="none",
            facecolor = "#2B2B2B",
            dpi=600)